In [214]:
import pandas as pd
pd.set_option('display.width', 1000)
import numpy as np
from datetime import timedelta


CHECK LINE 20 FOR WHAT TO DO NEXT
CONVERTING EXIT DATE INTO AGE3DATE IF NECESSARY

In [176]:
exit_notes_df = pd.read_excel('..\Data\TEIS Exit Data for NSS COPY.xlsx', sheet_name='Exit Notes')

In [210]:
#exit_notes_df = exit_notes_df.dropna().reset_index(drop=True)
#exit_notes_df

reopened_df = exit_notes_df[exit_notes_df['Exit Note'].str.contains('reopened', case=False, na=False)].reset_index(drop=True)
reopened_df = reopened_df.drop(columns=['POE', 'Notification Date', 'Initial Eligibility Date', 'Latest IFSP Date'])
reopened_df.head(20)

,Child ID,DOB,Initial IFSP Date,Exit Reason,Exit Date,Days from I-IFSP to Exit,Exit Note
0,473474,2017-11-05,2020-05-06,618 - Part B eligibility not determined,2020-11-04,182,**LEA county on the Transition page and demogr...
1,469243,2019-07-26,2019-10-02,618 - Parent withdraw,2021-02-11,498,02-11-2021 – PWN signed and returned for volun...
2,436339,2018-04-05,2019-02-13,618 - Part B eligibility not determined,2021-04-01,778,04/01/2021- File closed business day prior to ...
3,474102,2018-05-11,2020-04-09,618 - Part B eligibility not determined,2021-05-10,396,05/10/2021 TPC held then parents withdrew from...
4,476958,2018-10-17,2020-08-24,618 - Parent withdraw,2021-05-25,274,05/25/21- parent withdraw Reopened: 08/04/2020...
5,449017,2018-09-18,2018-12-12,618 - Attempts to contact unsuccessful,2021-06-01,902,06/01/2021- File closed after 10-day waiting p...
6,468728,2017-08-18,2020-02-14,618 - Part B eligible,2020-08-17,185,08/17/2020- File closed. Child determined Part...
7,469200,2019-07-13,2019-11-13,618 - Attempts to contact unsuccessful,2020-09-22,314,09/22/2020- unable to contact Reopened: 08/26/...
8,477102,2018-03-12,2020-04-14,618 - Parent withdraw,2021-01-20,281,1/20/2021- Parent withdraw from program due to...
9,449847,2017-11-02,2019-03-26,618 - Attempts to contact unsuccessful,2020-10-30,584,10/30/2020- File closed due to child turning 3...


In [211]:
#reopened_df['Exit Note'].str.extract('(\d+/)')
reopened_df['Exit Note'].str.findall('(\\d{2}/\\d{2}/\\d{4})')
dates_df = reopened_df['Exit Note'].str.findall('(\\d{2}/\\d{2}/\\d{4})')
dates_df = dates_df.to_frame()

dates_df['total_dates'] = dates_df['Exit Note'].apply(lambda x: len(str(x).split(' ')))
#display(dates_df)

dates_df.total_dates.describe()



#display(reopened_df['Exit Note'].str.findall('Reopened: \\d{2}/\\d{2}/\\d{4}').head(20))
#display(reopened_df['Exit Note'].str.findall('Exit Date: \\d{2}/\\d{2}/\\d{4}').head(20))

reopened_df['reopened_dates'] = reopened_df['Exit Note'].str.findall('Reopened: \\d{2}/\\d{2}/\\d{4}')
reopened_df['exit_dates'] = reopened_df['Exit Note'].str.findall('Exit Date: \\d{2}/\\d{2}/\\d{4}')

reopen_date_df = reopened_df.reopened_dates.to_frame().astype(str)
exit_date_df = reopened_df.exit_dates.to_frame().astype(str)

reopen_date_df = reopen_date_df.reopened_dates.str.split(',', expand=True)
reopen_date_df.columns = ['reopen1', 'reopen2', 'reopen3', 'reopen4', 'reopen5', 'reopen6']
exit_date_df = exit_date_df.exit_dates.str.split(',', expand=True)
exit_date_df.columns = ['exit1', 'exit2', 'exit3', 'exit4', 'exit5', 'exit6']

cols_r = ['reopen1', 'reopen2', 'reopen3', 'reopen4', 'reopen5', 'reopen6']
cols_e = ['exit1', 'exit2', 'exit3', 'exit4', 'exit5', 'exit6']

reopen_date_df[cols_r] = reopen_date_df[cols_r].replace({'\[':'', '\]':'', '\'':'', 'Reopened:':''}, regex=True)
exit_date_df[cols_e] = exit_date_df[cols_e].replace({'\[':'', '\]':'', '\'':'', 'Exit Date:':''}, regex=True)
display(reopen_date_df)
display(exit_date_df)
reopened_df['exit_dates'].loc[reopened_df['Child ID']==433605].explode()

#df['Address'].str.split(',', expand=True)
#display(reopened_df['Exit Note'].str.findall('Reopened: (\\d{2}/\\d{2}/\\d{4})').head(20))
#display(reopened_df['Exit Note'].str.findall('Exit Date: (\\d{2}/\\d{2}/\\d{4})').head(20))

,reopen1,reopen2,reopen3,reopen4,reopen5,reopen6
0,04/06/2020,None,None,None,None,None
1,02/05/2021,None,None,None,None,None
2,01/11/2019,None,None,None,None,None
3,02/28/2020,None,None,None,None,None
4,08/04/2020,None,None,None,None,None
...,...,...,...,...,...,...
1725,07/28/2020,None,None,None,None,None
1726,02/05/2019,None,None,None,None,None
1727,03/05/2020,None,None,None,None,None
1728,10/25/2018,None,None,None,None,None


,exit1,exit2,exit3,exit4,exit5,exit6
0,11/26/2019,None,None,None,None,None
1,04/07/2020,None,None,None,None,None
2,06/17/2018,None,None,None,None,None
3,12/05/2019,None,None,None,None,None
4,03/09/2020,None,None,None,None,None
...,...,...,...,...,...,...
1725,04/26/2020,None,None,None,None,None
1726,10/23/2018,None,None,None,None,None
1727,09/17/2019,None,None,None,None,None
1728,05/24/2018,None,None,None,None,None


132    Exit Date: 12/18/2020
132    Exit Date: 07/27/2020
132    Exit Date: 05/20/2019
132    Exit Date: 11/10/2018
Name: exit_dates, dtype: object

In [246]:
reopen_exit_date_df = reopen_date_df.join(exit_date_df)

cols_r_e =  ['reopen1', 'reopen2', 'reopen3', 'reopen4', 'reopen5', 'reopen6',
             'exit1', 'exit2', 'exit3', 'exit4', 'exit5', 'exit6']
reopen_exit_date_df = reopen_exit_date_df.astype('datetime64')
reopen_exit_date_df = reopen_exit_date_df.reindex(columns=['exit1', 'reopen1', 'exit2', 'reopen2', 'exit3', 'reopen3', 'exit4', 'reopen4', 'exit5', 'reopen5', 'exit6', 'reopen6'])
reopen_exit_date_df

ifsp_date = reopened_df[['Child ID', 'DOB', 'Initial IFSP Date', 'Exit Date', 'Days from I-IFSP to Exit']]
ifsp_date[['Initial IFSP Date', 'Exit Date']].astype('datetime64')
ifsp_date['age3_date'] = ifsp_date['DOB'] + pd.offsets.DateOffset(years=3)
reopen_exit_date_df = ifsp_date.join(reopen_exit_date_df)

###IF REOPEN DATE IS AFTER 'EXIT DATE', CHANGE 'EXIT DATE' TO CURRENT DAY AND RECALCULATE 'DAYS FROM I-IFSP TO EXIT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen1 > reopen_exit_date_df['Exit Date'], ['Exit Date']] = pd.to_datetime('today').normalize()



reopen_exit_date_df.loc[reopen_exit_date_df['Exit Date'] > reopen_exit_date_df['age3_date'], ['Exit Date']] = reopen_exit_date_df['age3_date']
CHECK TO CONVERT BACK TO DATETIME FROM STRING




#reopen_exit_date_df['Exit Date'] = reopen_exit_date_df['Exit Date'].astype('datetime64')

#reopen_exit_date_df['Exit Date'] - reopen_exit_date_df['DOB']


#reopen_exit_date_df['Days from I-IFSP to Exit'] = reopen_exit_date_df['Exit Date'] - reopen_exit_date_df['Initial IFSP Date']
#reopen_exit_date_df['Days from I-IFSP to Exit'] = reopen_exit_date_df['Days from I-IFSP to Exit'].dt.days



reopen_exit_date_df.loc[reopen_exit_date_df.exit1 < reopen_exit_date_df['Initial IFSP Date'], ['exit1']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.exit2 < reopen_exit_date_df['Initial IFSP Date'], ['exit2']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.exit3 < reopen_exit_date_df['Initial IFSP Date'], ['exit3']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.exit4 < reopen_exit_date_df['Initial IFSP Date'], ['exit4']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.exit5 < reopen_exit_date_df['Initial IFSP Date'], ['exit5']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.exit6 < reopen_exit_date_df['Initial IFSP Date'], ['exit6']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen1 < reopen_exit_date_df['Initial IFSP Date'], ['reopen1']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen2 < reopen_exit_date_df['Initial IFSP Date'], ['reopen2']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen3 < reopen_exit_date_df['Initial IFSP Date'], ['reopen3']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen4 < reopen_exit_date_df['Initial IFSP Date'], ['reopen4']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen5 < reopen_exit_date_df['Initial IFSP Date'], ['reopen5']] = 'NaT'
reopen_exit_date_df.loc[reopen_exit_date_df.reopen6 < reopen_exit_date_df['Initial IFSP Date'], ['reopen6']] = 'NaT'

reopen_exit_date_df = reopen_exit_date_df[reopen_exit_date_df.exit1.notnull()].reset_index(drop=True)

#reopen_exit_date_df.exit2 = np.where(reopen_exit_date_df.exit2 == reopen_exit_date_df['Exit Date'], reopen_exit_date_df.exit3, reopen_exit_date_df.exit2)
reopen_exit_date_df.exit1 = np.where(reopen_exit_date_df.exit1 == reopen_exit_date_df['Exit Date'], reopen_exit_date_df.exit2, reopen_exit_date_df.exit1)

#reopen_exit_date_df.exit1 = reopen_exit_date_df.exit1.astype('datetime64').dt.date
#reopen_exit_date_df.reopen1 = reopen_exit_date_df.reopen1.astype('datetime64').dt.date

reopen_exit_date_df['r1-e1'] = reopen_exit_date_df.reopen1 - reopen_exit_date_df.exit1
reopen_exit_date_df['r2-e2'] = reopen_exit_date_df.reopen2 - reopen_exit_date_df.exit2
reopen_exit_date_df['r3-e3'] = reopen_exit_date_df.reopen3 - reopen_exit_date_df.exit3
reopen_exit_date_df['r4-e4'] = reopen_exit_date_df.reopen4 - reopen_exit_date_df.exit4
reopen_exit_date_df['r5-e5'] = reopen_exit_date_df.reopen5 - reopen_exit_date_df.exit5
reopen_exit_date_df['r6-e6'] = reopen_exit_date_df.reopen6 - reopen_exit_date_df.exit6
reopen_exit_date_df.dtypes

reopen_exit_date_df['r1-e1'] = reopen_exit_date_df['r1-e1'].fillna(pd.Timedelta(seconds=0))
reopen_exit_date_df['r2-e2'] = reopen_exit_date_df['r2-e2'].fillna(pd.Timedelta(seconds=0))
reopen_exit_date_df['r3-e3'] = reopen_exit_date_df['r3-e3'].fillna(pd.Timedelta(seconds=0))
reopen_exit_date_df['r4-e4'] = reopen_exit_date_df['r4-e4'].fillna(pd.Timedelta(seconds=0))
reopen_exit_date_df['r5-e5'] = reopen_exit_date_df['r5-e5'].fillna(pd.Timedelta(seconds=0))
reopen_exit_date_df['r6-e6'] = reopen_exit_date_df['r6-e6'].fillna(pd.Timedelta(seconds=0))

reopen_exit_date_df['day_diff'] = reopen_exit_date_df['r1-e1'] + reopen_exit_date_df['r2-e2'] + reopen_exit_date_df['r3-e3'] + reopen_exit_date_df['r4-e4'] + reopen_exit_date_df['r5-e5'] + reopen_exit_date_df['r6-e6']
reopen_exit_date_df.day_diff = reopen_exit_date_df.day_diff.astype('timedelta64[D]').astype(int)
reopen_exit_date_df['actual_service_time'] = reopen_exit_date_df['Days from I-IFSP to Exit'] - reopen_exit_date_df.day_diff
display(reopen_exit_date_df['actual_service_time'].sort_values())
display(reopen_exit_date_df.iloc[563])
reopen_exit_date_df.dtypes


###if exit1 = final exit date, repopulate exit1 with exit2

C:\Users\Alex7\AppData\Local\Temp/ipykernel_10844/1792512479.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ifsp_date['age3_date'] = ifsp_date['DOB'] + pd.offsets.DateOffset(years=3)


358    -179
247     -81
394      12
130      13
81       21
       ... 
618    1001
546    1007
194    1029
563    1030
607    1056
Name: actual_service_time, Length: 723, dtype: int64

Child ID                                 423973
DOB                         2017-09-07 00:00:00
Initial IFSP Date           2017-10-17 00:00:00
Exit Date                   1599436800000000000
Days from I-IFSP to Exit                   1055
age3_date                   2020-09-07 00:00:00
exit1                       2020-09-06 00:00:00
reopen1                     2020-10-01 00:00:00
exit2                                       NaT
reopen2                                     NaT
exit3                                       NaT
reopen3                                     NaT
exit4                                       NaT
reopen4                                     NaT
exit5                                       NaT
reopen5                                     NaT
exit6                                       NaT
reopen6                                     NaT
r1-e1                          25 days 00:00:00
r2-e2                           0 days 00:00:00
r3-e3                           0 days 0

Child ID                              int64
DOB                          datetime64[ns]
Initial IFSP Date            datetime64[ns]
Exit Date                            object
Days from I-IFSP to Exit              int64
age3_date                    datetime64[ns]
exit1                        datetime64[ns]
reopen1                      datetime64[ns]
exit2                        datetime64[ns]
reopen2                      datetime64[ns]
exit3                        datetime64[ns]
reopen3                      datetime64[ns]
exit4                        datetime64[ns]
reopen4                      datetime64[ns]
exit5                        datetime64[ns]
reopen5                      datetime64[ns]
exit6                        datetime64[ns]
reopen6                      datetime64[ns]
r1-e1                       timedelta64[ns]
r2-e2                       timedelta64[ns]
r3-e3                       timedelta64[ns]
r4-e4                       timedelta64[ns]
r5-e5                       time

In [204]:
#reopen_exit_date_df.exit1 = np.where(reopen_exit_date_df.exit1 == reopen_exit_date_df['Exit Date'], reopen_exit_date_df.exit1, reopen_exit_date_df.exit2)
#reopen_exit_date_df.exit1 = reopen_exit_date_df.exit1.astype('datetime64').dt.date

#reopen_exit_date_df.loc[reopen_exit_date_df.exit2 < reopen_exit_date_df['Initial IFSP Date'], ['exit2']] = 'NaT'
#reopen_exit_date_df.loc[reopen_exit_date_df.exit3 < reopen_exit_date_df['Initial IFSP Date'], ['exit3']] = 'NaT'
#reopen_exit_date_df.loc[reopen_exit_date_df.exit4 < reopen_exit_date_df['Initial IFSP Date'], ['exit4']] = 'NaT'
#reopen_exit_date_df.loc[reopen_exit_date_df.exit5 < reopen_exit_date_df['Initial IFSP Date'], ['exit5']] = 'NaT'
#reopen_exit_date_df.loc[reopen_exit_date_df.exit6 < reopen_exit_date_df['Initial IFSP Date'], ['exit6']] = 'NaT'
display(reopen_exit_date_df.iloc[67])

Child ID                                 433605
Initial IFSP Date           2018-05-23 00:00:00
Exit Date                   2022-03-17 00:00:00
Days from I-IFSP to Exit                   1394
exit1                       2020-12-18 00:00:00
reopen1                     2021-01-05 00:00:00
exit2                       2020-07-27 00:00:00
reopen2                     2020-11-02 00:00:00
exit3                       2019-05-20 00:00:00
reopen3                     2020-07-09 00:00:00
exit4                       2018-11-10 00:00:00
reopen4                     2018-12-17 00:00:00
exit5                                       NaT
reopen5                                     NaT
exit6                                       NaT
reopen6                                     NaT
r1-e1                          18 days 00:00:00
r2-e2                          98 days 00:00:00
r3-e3                         416 days 00:00:00
r4-e4                          37 days 00:00:00
r5-e5                           0 days 0